In [1]:
import pandas as pd
import time
import re
import numpy as np
import requests

In [2]:
df = pd.read_excel("OCA Data.xlsx")
# df = pd.read_csv("real data lengkap.csv")
df

,First Index,Interaction ID,Channel,Last room created,Last room resolved,Current room created,Current room resolved,Assignee Agent Name,Assignee Agent Department,Resolved Agent Name,Resolved Agent Department,Conversation,Produk,Symptom,Type,Summary,Status Penyelesaian,Kesesuaian,Eligible Chatbot
0,47,113715,Whatsapp,-,~,2024-10-01 08:50,2024-10-02 09:46,Fidlin Aulia,Assurance Product Operation - OCA,Fidlin Aulia,Assurance Product Operation - OCA,"""customer : 3""\n\nagent : mohon tunggu, oca re...",oca,oca->pelanggan tidak bisa login,Incident,"Pelanggan, Arief dari Humas Universitas Tadula...",Terselesaikan,Sesuai,No
1,57,113726,Whatsapp,2024-09-30 21:56,2024-09-30 22:20,2024-10-01 09:04,2024-10-01 11:25,Rezi Eka Putri,Assurance Product Operation - OCA,Rezi Eka Putri,Assurance Product Operation - OCA,"""customer : 3""\n\nagent : mohon tunggu, oca re...",oca,oca->terdapat kegagalan variable sehingga meny...,Request,"Dalam percakapan ini, pelanggan, Bapak Ferry, ...",Sebagian Terselesaikan,Tidak Sesuai,No
2,181,113377,Whatsapp,-,~,2024-10-01 13:19,2024-10-01 13:45,Nimatul HA,Grup 2,Nimatul HA,Grup 2,"""customer : 9""\n\nagent : mohon menunggu sesaa...",oca,oca->user not response,Information,Pelanggan menghubungi agen untuk meminta dafta...,Belum Terselesaikan,Sesuai,No
3,204,113231,Email,2024-09-30 14:37,2024-09-30 14:41,2024-10-01 14:08,2024-10-01 14:09,Danang Aristya Wiebowo,Grup 1,Danang Aristya Wiebowo,Grup 1,"""customer : dear mba rindy,\n\n\n\n\nberikut y...",oca,oca->template/template code tidak sesuai,Request,"Pelanggan, Panji Kukuh Priambodho, menghadapi ...",Sebagian Terselesaikan,Sesuai,No
4,300,113252,Whatsapp,-,~,2024-10-01 19:17,2024-10-01 20:53,Nofvel Maulana,Grup 2,Nofvel Maulana,Grup 2,"""customer : cs""\n\nagent : mohon menunggu sesa...",oca,oca->info layanan oca,Information,"Pelanggan, Syadewo Putra dari Royal King, meng...",Sebagian Terselesaikan,Tidak Sesuai,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505,31159,113856,Whatsapp,-,~,2024-12-27 14:19,2024-12-27 14:46,Anggita Windiana Sari,Grup 2,Anggita Windiana Sari,Grup 2,"""customer : pengajuan layanan oca""\n\nagent : ...",oca,t-money->salah sambung,Incident,"Pelanggan, Sardani, melaporkan masalah terkait...",Belum Terselesaikan,Sesuai,No
506,31553,113255,Whatsapp,-,~,2024-12-28 19:43,2024-12-28 19:59,Sesep Irawan,Grup 2,Sesep Irawan,Grup 2,"""customer : pengajuan layanan oca""\n\nagent : ...",oca,t-money->user tidak merespon,Request,Pelanggan menghubungi layanan OCA namun tidak ...,Belum Terselesaikan,Tidak Sesuai,No
507,31678,113803,Whatsapp,-,~,2024-12-29 15:07,2024-12-29 15:13,Budi Yulianto,Grup 2,Budi Yulianto,Grup 2,"""customer : pengajuan layanan oca""\n\nagent : ...",oca,tidak jelas,Anomaly,"Dalam percakapan ini, pelanggan tidak mengungk...",tidak jelas,tidak jelas,No
508,31951,113114,Whatsapp,-,~,2024-12-30 11:49,2024-12-30 11:58,Dinda Sukma Wardani,Grup 2,Dinda Sukma Wardani,Grup 2,"""customer : pengajuan layanan oca""\n\nagent : ...",oca,tidak jelas,Complaint,"Dalam percakapan ini, pelanggan, Ibu Heny, men...",tidak jelas,tidak jelas,No


In [ ]:
# df = df.iloc[0:25].reset_index(drop=True)
# df

In [3]:
# Konfigurasi API LLAMA Telkom AI
url = "[url]"
headers = {
    'accept': 'application/json',
    'x-api-key': "[api key]",
    'Content-Type': 'application/json'
}

In [4]:
def generate_question_answer(text):
    """
    Fungsi untuk mengidentifikasi satu pertanyaan dan jawaban yang paling relevan dari percakapan pelanggan menggunakan API LLAMA31.
    """
    prompt = f"""
    ROLE: 
    Kamu adalah Analis Customer Service Profesional yang bertugas menganalisis percakapan antara customer dan agent untuk mengekstrak pertanyaan dari customer serta jawaban yang diberikan oleh agent.
    
    INSTRUCTION: 
    Analisis dan ekstrak pertanyaan dari customer dan jawaban dari agent dari percakapan berikut '{text}'. Berdasarkan instruksi berikut:
    1. Pertanyaan berkaitan dengan produk, fitur, kebijakan, informasi layanan, atau sistem secara umum
    2. Pertanyaan bisa dijawab oleh AI/chatbot tanpa perlu informasi spesifik pelanggan
    3. Pertanyaan tidak menanyakan data real-time, status pesanan, tiket, status email masuk, status gangguan aplikasi, atau akses ke sistem internal.
    4. Jawaban yang diberikan agent sesuai untuk pertanyaan tersebut
    5. Pertanyaan berkaitan dengan cara melakukan sesuatu atau prosedur
    7. Pertanyaan tidak memerlukan jawaban/tindakan manual oleh agen/manusia
    8. Jawaban tidak membutuhkan data pribadi customer. Seperti meminta mengisi data diri, nomor telepon, email, alamat, dll. 
    9. Jawaban tidak meminta customer mununggu untuk info/proses lebih lanjut atau menunggu balasan dari pihak lain atau melakukan pengecekan atau meminta konfirmasi. (contoh: "Silakan tunggu/Mohon tunggu", "Kami akan melakukan pengecekan", "Mohon konfirmasi")
    10. Pertanyaan dan jawabannya tidak mengandung perihal nomor tiket (DOA) atau invoice (INV) atau nomor pesanan (PO) atau nomor pelanggan (CIF) atau LOP atau nomor lainnya.
    11. Pertanyaan tidak menanyakan hal mengenai status diri sendiri (seperti: "Bagaimana saya bisa membayar tagihan saya?", "Apakah saya sudah melakukan pembayaran tagihan saya?")
    12. Jawaban tidak meminta capture atau screenshoot atau meminta informasi yang hanya bisa dilihat di layar customer
    13. Jawaban tidak memberikan pernyataan bahwa agent telah melakukan tindakan tertentu atau telah menerima informasi tertentu dari customer. (contoh: "Saya telah mengirimkan email kepada Anda", "Kami telah menerima perimintaan Anda ...", "Kami telah membantu Anda ...")
    
    SCENARIO:
    Kamu akan menganalisis percakapan antara customer dan agent, lalu ekstrak pertanyaan dan jawaban dari percakapan yang sesuai dengan instruksi di atas.
    Jika percakapan tidak berisi pertanyaan atau jawaban yang jelas dan tidak sesuai instruksi, berikan "No data".
    Jika agent menyebutkan Nama orang atau seperti "Pak/Bu" pada jawaban, HAPUS nama atau ganti dengan "Anda" lalu sesuaikan kembali jawaban.
    Jika pertanyaan adalah tata cara berikan secara step by step tetapi tetap singkat dan jelas.

    EXTRACT:
    Pertanyaan: [Tuliskan hanya SATU pertanyaan dari customer yang sesuai instruksi dalam bentuk kalimat tanya yang jelas]
    Jawaban: [Tuliskan hanya SATU jawaban dari agent yang sesuai instruksi secara singkat, padat, jelas, dan mudah dipahami]
    """
    
    payload = {
        "message": [{"role": "user", "content": prompt}]
    }
    
    try:
        response = requests.post(url, headers=headers, json=payload, timeout=300)
        response_data = response.json().get("result", "")
        print(f"Output: {response_data}\n")
        
        if "Pertanyaan:" in response_data and "Jawaban:" in response_data:
            parts = response_data.split("Jawaban:")
            pertanyaan = parts[0].replace("Pertanyaan:", "").strip()
            jawaban = parts[1].strip() if len(parts) > 1 else "No data"
        else:
            pertanyaan, jawaban = "No data", "No data"
        
        return pertanyaan, jawaban
    
    except Exception as e:
        print(f"Error generating question-answer: {e}")
        return "Error", "Error"

def process_conversations_llama31(df, batch_size=1000, sleep_time=60):
    """
    Fungsi untuk memproses percakapan dan mengekstrak 'Pertanyaan' dan 'Jawaban' menggunakan LLAMA31.
    """
    total_rows = len(df)
    
    if 'Pertanyaan' not in df.columns:
        df['Pertanyaan'] = ""
    if 'Jawaban' not in df.columns:
        df['Jawaban'] = ""

    for i in range(total_rows):
        try:
            text = df.loc[i, "Conversation"]
            if pd.isna(text) or text.strip() == "":
                df.loc[i, 'Pertanyaan'] = "No data"
                df.loc[i, 'Jawaban'] = "No data"
                continue
            
            pertanyaan, jawaban = generate_question_answer(text)
            df.loc[i, 'Pertanyaan'] = pertanyaan
            df.loc[i, 'Jawaban'] = jawaban
        
        except Exception as e:
            print(f"[{i}]> Error: {e}")
            df.loc[i, 'Pertanyaan'] = "Error"
            df.loc[i, 'Jawaban'] = "Error"
        
        if (i + 1) % batch_size == 0:
            print(f"Processed {i + 1}/{total_rows} rows. Sleeping for {sleep_time} seconds...")
            time.sleep(sleep_time)
    
    return df

In [5]:
%%time
df = process_conversations_llama31(df)
df

Output: Setelah menganalisis percakapan antara customer dan agent, saya menemukan beberapa pertanyaan dan jawaban yang sesuai dengan instruksi di atas. Berikut adalah hasil ekstraksi:

1. Pertanyaan: Bagaimana cara melakukan blast ke 220-240 nomor per hari agar limit jumlah maksimal blastnya bisa naik ke 1000/ hari dan seterusnya?
Jawaban: Anda dapat melakukan blast dengan jumlah yang lebih besar dengan membatasi jumlah blast per hari menjadi 220-240. Dengan demikian, limit jumlah maksimal blast Anda akan meningkat ke 1000/hari dan seterusnya.

2. Pertanyaan: Bagaimana cara melihat tutorial penggunaan dashboard di menu getting started?
Jawaban: Anda dapat melihat tutorial penggunaan dashboard di menu getting started yang ada pada dashboard Anda.

3. Pertanyaan: Apakah ada hal lain yang ingin ditanyakan?
Jawaban: Tidak, tidak ada lagi pertanyaan dari saya.

4. Pertanyaan: Bagaimana cara mengakses menu getting started?
Jawaban: Menu getting started dapat diakses pada dashboard Anda.

5. 

,First Index,Interaction ID,Channel,Last room created,Last room resolved,Current room created,Current room resolved,Assignee Agent Name,Assignee Agent Department,Resolved Agent Name,...,Conversation,Produk,Symptom,Type,Summary,Status Penyelesaian,Kesesuaian,Eligible Chatbot,Pertanyaan,Jawaban
0,47,113715,Whatsapp,-,~,2024-10-01 08:50,2024-10-02 09:46,Fidlin Aulia,Assurance Product Operation - OCA,Fidlin Aulia,...,"""customer : 3""\n\nagent : mohon tunggu, oca re...",oca,oca->pelanggan tidak bisa login,Incident,"Pelanggan, Arief dari Humas Universitas Tadula...",Terselesaikan,Sesuai,No,Setelah menganalisis percakapan antara custome...,Anda dapat melakukan blast dengan jumlah yang ...
1,57,113726,Whatsapp,2024-09-30 21:56,2024-09-30 22:20,2024-10-01 09:04,2024-10-01 11:25,Rezi Eka Putri,Assurance Product Operation - OCA,Rezi Eka Putri,...,"""customer : 3""\n\nagent : mohon tunggu, oca re...",oca,oca->terdapat kegagalan variable sehingga meny...,Request,"Dalam percakapan ini, pelanggan, Bapak Ferry, ...",Sebagian Terselesaikan,Tidak Sesuai,No,Setelah menganalisis percakapan antara custome...,"Baik sebelumnya mohon maaf, jika berkenan sila..."
2,181,113377,Whatsapp,-,~,2024-10-01 13:19,2024-10-01 13:45,Nimatul HA,Grup 2,Nimatul HA,...,"""customer : 9""\n\nagent : mohon menunggu sesaa...",oca,oca->user not response,Information,Pelanggan menghubungi agen untuk meminta dafta...,Belum Terselesaikan,Sesuai,No,Setelah menganalisis percakapan antara custome...,"** Selamat siang Bapak/Ibu, terima kasih telah..."
3,204,113231,Email,2024-09-30 14:37,2024-09-30 14:41,2024-10-01 14:08,2024-10-01 14:09,Danang Aristya Wiebowo,Grup 1,Danang Aristya Wiebowo,...,"""customer : dear mba rindy,\n\n\n\n\nberikut y...",oca,oca->template/template code tidak sesuai,Request,"Pelanggan, Panji Kukuh Priambodho, menghadapi ...",Sebagian Terselesaikan,Sesuai,No,Bagaimana saya bisa mendaftar untuk layanan Anda?,"Untuk mendaftar, silakan kunjungi situs web ka..."
4,300,113252,Whatsapp,-,~,2024-10-01 19:17,2024-10-01 20:53,Nofvel Maulana,Grup 2,Nofvel Maulana,...,"""customer : cs""\n\nagent : mohon menunggu sesa...",oca,oca->info layanan oca,Information,"Pelanggan, Syadewo Putra dari Royal King, meng...",Sebagian Terselesaikan,Tidak Sesuai,No,Setelah menganalisis percakapan antara custome...,"Baik sebelumnya mohon maaf, jika berkenan sila..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505,31159,113856,Whatsapp,-,~,2024-12-27 14:19,2024-12-27 14:46,Anggita Windiana Sari,Grup 2,Anggita Windiana Sari,...,"""customer : pengajuan layanan oca""\n\nagent : ...",oca,t-money->salah sambung,Incident,"Pelanggan, Sardani, melaporkan masalah terkait...",Belum Terselesaikan,Sesuai,No,Setelah menganalisis percakapan antara custome...,Mohon maaf saat ini Anda terhubung dengan laya...
506,31553,113255,Whatsapp,-,~,2024-12-28 19:43,2024-12-28 19:59,Sesep Irawan,Grup 2,Sesep Irawan,...,"""customer : pengajuan layanan oca""\n\nagent : ...",oca,t-money->user tidak merespon,Request,Pelanggan menghubungi layanan OCA namun tidak ...,Belum Terselesaikan,Tidak Sesuai,No,Setelah menganalisis percakapan antara custome...,"""mohon tunggu, oca representative akan segera ..."
507,31678,113803,Whatsapp,-,~,2024-12-29 15:07,2024-12-29 15:13,Budi Yulianto,Grup 2,Budi Yulianto,...,"""customer : pengajuan layanan oca""\n\nagent : ...",oca,tidak jelas,Anomaly,"Dalam percakapan ini, pelanggan tidak mengungk...",tidak jelas,tidak jelas,No,Setelah menganalisis percakapan antara custome...,"**\nBaik bapak/ibu, apabila tidak ada yang dap..."
508,31951,113114,Whatsapp,-,~,2024-12-30 11:49,2024-12-30 11:58,Dinda Sukma Wardani,Grup 2,Dinda Sukma Wardani,...,"""customer : pengajuan layanan oca""\n\nagent : ...",oca,tidak jelas,Complaint,"Dalam percakapan ini, pelanggan, Ibu Heny, men...",tidak jelas,tidak jelas,No,Setelah menganalisis percakapan antara custome...,Benar Anda.\n\n2. Pertanyaan: Kamu cowok gak j...


In [8]:
df.to_csv("oca_test1.csv", index=False)

In [9]:
df

,First Index,Interaction ID,Channel,Last room created,Last room resolved,Current room created,Current room resolved,Assignee Agent Name,Assignee Agent Department,Resolved Agent Name,...,Conversation,Produk,Symptom,Type,Summary,Status Penyelesaian,Kesesuaian,Eligible Chatbot,Pertanyaan,Jawaban
0,47,113715,Whatsapp,-,~,2024-10-01 08:50,2024-10-02 09:46,Fidlin Aulia,Assurance Product Operation - OCA,Fidlin Aulia,...,"""customer : 3""\n\nagent : mohon tunggu, oca re...",oca,oca->pelanggan tidak bisa login,Incident,"Pelanggan, Arief dari Humas Universitas Tadula...",Terselesaikan,Sesuai,No,Setelah menganalisis percakapan antara custome...,Anda dapat melakukan blast dengan jumlah yang ...
1,57,113726,Whatsapp,2024-09-30 21:56,2024-09-30 22:20,2024-10-01 09:04,2024-10-01 11:25,Rezi Eka Putri,Assurance Product Operation - OCA,Rezi Eka Putri,...,"""customer : 3""\n\nagent : mohon tunggu, oca re...",oca,oca->terdapat kegagalan variable sehingga meny...,Request,"Dalam percakapan ini, pelanggan, Bapak Ferry, ...",Sebagian Terselesaikan,Tidak Sesuai,No,Setelah menganalisis percakapan antara custome...,"Baik sebelumnya mohon maaf, jika berkenan sila..."
2,181,113377,Whatsapp,-,~,2024-10-01 13:19,2024-10-01 13:45,Nimatul HA,Grup 2,Nimatul HA,...,"""customer : 9""\n\nagent : mohon menunggu sesaa...",oca,oca->user not response,Information,Pelanggan menghubungi agen untuk meminta dafta...,Belum Terselesaikan,Sesuai,No,Setelah menganalisis percakapan antara custome...,"** Selamat siang Bapak/Ibu, terima kasih telah..."
3,204,113231,Email,2024-09-30 14:37,2024-09-30 14:41,2024-10-01 14:08,2024-10-01 14:09,Danang Aristya Wiebowo,Grup 1,Danang Aristya Wiebowo,...,"""customer : dear mba rindy,\n\n\n\n\nberikut y...",oca,oca->template/template code tidak sesuai,Request,"Pelanggan, Panji Kukuh Priambodho, menghadapi ...",Sebagian Terselesaikan,Sesuai,No,Bagaimana saya bisa mendaftar untuk layanan Anda?,"Untuk mendaftar, silakan kunjungi situs web ka..."
4,300,113252,Whatsapp,-,~,2024-10-01 19:17,2024-10-01 20:53,Nofvel Maulana,Grup 2,Nofvel Maulana,...,"""customer : cs""\n\nagent : mohon menunggu sesa...",oca,oca->info layanan oca,Information,"Pelanggan, Syadewo Putra dari Royal King, meng...",Sebagian Terselesaikan,Tidak Sesuai,No,Setelah menganalisis percakapan antara custome...,"Baik sebelumnya mohon maaf, jika berkenan sila..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505,31159,113856,Whatsapp,-,~,2024-12-27 14:19,2024-12-27 14:46,Anggita Windiana Sari,Grup 2,Anggita Windiana Sari,...,"""customer : pengajuan layanan oca""\n\nagent : ...",oca,t-money->salah sambung,Incident,"Pelanggan, Sardani, melaporkan masalah terkait...",Belum Terselesaikan,Sesuai,No,Setelah menganalisis percakapan antara custome...,Mohon maaf saat ini Anda terhubung dengan laya...
506,31553,113255,Whatsapp,-,~,2024-12-28 19:43,2024-12-28 19:59,Sesep Irawan,Grup 2,Sesep Irawan,...,"""customer : pengajuan layanan oca""\n\nagent : ...",oca,t-money->user tidak merespon,Request,Pelanggan menghubungi layanan OCA namun tidak ...,Belum Terselesaikan,Tidak Sesuai,No,Setelah menganalisis percakapan antara custome...,"""mohon tunggu, oca representative akan segera ..."
507,31678,113803,Whatsapp,-,~,2024-12-29 15:07,2024-12-29 15:13,Budi Yulianto,Grup 2,Budi Yulianto,...,"""customer : pengajuan layanan oca""\n\nagent : ...",oca,tidak jelas,Anomaly,"Dalam percakapan ini, pelanggan tidak mengungk...",tidak jelas,tidak jelas,No,Setelah menganalisis percakapan antara custome...,"**\nBaik bapak/ibu, apabila tidak ada yang dap..."
508,31951,113114,Whatsapp,-,~,2024-12-30 11:49,2024-12-30 11:58,Dinda Sukma Wardani,Grup 2,Dinda Sukma Wardani,...,"""customer : pengajuan layanan oca""\n\nagent : ...",oca,tidak jelas,Complaint,"Dalam percakapan ini, pelanggan, Ibu Heny, men...",tidak jelas,tidak jelas,No,Setelah menganalisis percakapan antara custome...,Benar Anda.\n\n2. Pertanyaan: Kamu cowok gak j...


In [12]:
# Filter the DataFrame for rows where 'Pertanyaan' or 'Jawaban' is "Error"
error_rows = df[(df['Pertanyaan'] == "Error") | (df['Jawaban'] == "Error")]
    
    # Display the filtered DataFrame
error_rows

,First Index,Interaction ID,Channel,Last room created,Last room resolved,Current room created,Current room resolved,Assignee Agent Name,Assignee Agent Department,Resolved Agent Name,...,Conversation,Produk,Symptom,Type,Summary,Status Penyelesaian,Kesesuaian,Eligible Chatbot,Pertanyaan,Jawaban


In [7]:
df = pd.read_csv("oca_test1.csv")
df

FileNotFoundError: [Errno 2] No such file or directory: 'oca_test1.csv'

In [10]:
def generate_question_answer(text):
    """
    Fungsi untuk mengidentifikasi satu pertanyaan dan jawaban yang paling relevan dari percakapan pelanggan menggunakan API LLAMA31.
    """
    prompt = f"""
    ROLE: 
    Kamu adalah Analis Customer Service Profesional yang bertugas menganalisis percakapan antara customer dan agent untuk mengekstrak pertanyaan dari customer serta jawaban yang diberikan oleh agent.
    
    INSTRUCTION: 
    Analisis dan ekstrak pertanyaan dari customer dan jawaban dari agent dari percakapan berikut '{text}'. Berdasarkan instruksi berikut:
    1. Pertanyaan berkaitan dengan produk, fitur, kebijakan, informasi layanan, atau sistem secara umum
    2. Pertanyaan bisa dijawab oleh AI/chatbot tanpa perlu informasi spesifik pelanggan
    3. Pertanyaan tidak menanyakan data real-time, status pesanan, tiket, status email masuk, status gangguan aplikasi, atau akses ke sistem internal.
    4. Jawaban yang diberikan agent sesuai untuk pertanyaan tersebut
    5. Pertanyaan berkaitan dengan cara melakukan sesuatu atau prosedur
    7. Pertanyaan tidak memerlukan jawaban/tindakan manual oleh agen/manusia
    8. Jawaban tidak membutuhkan data pribadi customer. Seperti meminta mengisi data diri, nomor telepon, email, alamat, dll. 
    9. Jawaban tidak meminta customer mununggu untuk info/proses lebih lanjut atau menunggu balasan dari pihak lain atau melakukan pengecekan atau meminta konfirmasi. (contoh: "Silakan tunggu/Mohon tunggu", "Kami akan melakukan pengecekan", "Mohon konfirmasi")
    10. Pertanyaan dan jawabannya tidak mengandung perihal nomor tiket (DOA) atau invoice (INV) atau nomor pesanan (PO) atau nomor pelanggan (CIF) atau LOP atau nomor lainnya.
    11. Pertanyaan tidak menanyakan hal mengenai status diri sendiri (seperti: "Bagaimana saya bisa membayar tagihan saya?", "Apakah saya sudah melakukan pembayaran tagihan saya?")
    12. Jawaban tidak meminta capture atau screenshoot atau meminta informasi yang hanya bisa dilihat di layar customer
    13. Jawaban tidak memberikan pernyataan bahwa agent telah melakukan tindakan tertentu atau telah menerima informasi tertentu dari customer. (contoh: "Saya telah mengirimkan email kepada Anda", "Kami telah menerima perimintaan Anda ...", "Kami telah membantu Anda ...")
    
    SCENARIO:
    Kamu akan menganalisis percakapan antara customer dan agent, lalu ekstrak pertanyaan dan jawaban dari percakapan yang sesuai dengan instruksi di atas.
    Jika percakapan tidak berisi pertanyaan atau jawaban yang jelas dan tidak sesuai instruksi, berikan "No data".
    Jika agent menyebutkan Nama orang atau seperti "Pak/Bu" pada jawaban, HAPUS nama atau ganti dengan "Anda" lalu sesuaikan kembali jawaban.
    Jika pertanyaan adalah tata cara berikan secara step by step tetapi tetap singkat dan jelas.

    EXTRACT:
    Pertanyaan: [Tuliskan hanya SATU pertanyaan dari customer yang sesuai instruksi dalam bentuk kalimat tanya yang jelas]
    Jawaban: [Tuliskan hanya SATU jawaban dari agent yang sesuai instruksi secara singkat, padat, jelas, dan mudah dipahami]
    """
    
    payload = {
        "message": [{"role": "user", "content": prompt}]
    }
    
    try:
        response = requests.post(url, headers=headers, json=payload, timeout=300)
        response_data = response.json().get("result", "")
        
        if "Pertanyaan:" in response_data and "Jawaban:" in response_data:
            parts = response_data.split("Jawaban:")
            pertanyaan = parts[0].replace("Pertanyaan:", "").strip()
            jawaban = parts[1].strip() if len(parts) > 1 else "No data"
        else:
            pertanyaan, jawaban = "No data", "No data"
        
        return pertanyaan, jawaban
    
    except Exception as e:
        print(f"Error generating question-answer: {e}")
        return "Error", "Error"


def process_conversations_llama31(df, batch_size=1000, sleep_time=60):
    """
    Fungsi untuk memproses percakapan dan mengekstrak 'Pertanyaan' dan 'Jawaban' menggunakan LLAMA31.
    """
    total_rows = len(df)
    
    if 'Pertanyaan' not in df.columns:
        df['Pertanyaan'] = ""
    if 'Jawaban' not in df.columns:
        df['Jawaban'] = ""

    for i in range(total_rows):
        try:
            text = df.loc[i, "Conversation"]
            if pd.isna(text) or text.strip() == "":
                df.loc[i, 'Pertanyaan'] = "No data"
                df.loc[i, 'Jawaban'] = "No data"
                continue
            
            pertanyaan, jawaban = generate_question_answer(text)
            df.loc[i, 'Pertanyaan'] = pertanyaan
            df.loc[i, 'Jawaban'] = jawaban
        
        except Exception as e:
            print(f"[{i}]> Error: {e}")
            df.loc[i, 'Pertanyaan'] = "Error"
            df.loc[i, 'Jawaban'] = "Error"
        
        if (i + 1) % batch_size == 0:
            print(f"Processed {i + 1}/{total_rows} rows. Sleeping for {sleep_time} seconds...")
            time.sleep(sleep_time)
    
    return df


def reprocess_errors(df, batch_size=1000, sleep_time=60):
    """
    Fungsi untuk memproses ulang hanya baris yang memiliki Error atau No data.
    """
    # Filter baris yang perlu diproses ulang
    error_df = df[(df['Pertanyaan'].isin(["Error"])) | (df['Jawaban'].isin(["Error"]))]
    total_rows = len(error_df)

    if total_rows == 0:
        print("Tidak ada data yang perlu diproses ulang.")
        return df

    print(f"Memproses ulang {total_rows} baris yang Error...")

    for index in error_df.index:
        try:
            text = df.loc[index, "Conversation"]
            if pd.isna(text) or text.strip() == "":
                df.loc[index, 'Pertanyaan'] = "No data"
                df.loc[index, 'Jawaban'] = "No data"
                continue
            
            pertanyaan, jawaban = generate_question_answer(text)
            df.loc[index, 'Pertanyaan'] = pertanyaan
            df.loc[index, 'Jawaban'] = jawaban
        
        except Exception as e:
            print(f"[{index}]> Error: {e}")
            df.loc[index, 'Pertanyaan'] = "Error"
            df.loc[index, 'Jawaban'] = "Error"
        
        if (index + 1) % batch_size == 0:
            print(f"Processed {index + 1}/{total_rows} rows. Sleeping for {sleep_time} seconds...")
            time.sleep(sleep_time)
    
    print("Proses ulang selesai.")
    return df


In [11]:
%%time
df = reprocess_errors(df)

Memproses ulang 15 baris yang Error...
Proses ulang selesai.
CPU times: user 147 ms, sys: 32.2 ms, total: 179 ms
Wall time: 1min 11s


In [17]:
df = pd.read_csv("mytens_test1.csv")
df

,First Index,Interaction ID,Channel,Last room created,Last room resolved,Current room created,Current room resolved,Last Inbound Date,Assignee Agent Name,Assignee Agent Department,...,Symptom,Type,Summary GPT4o,Status Penyelesaian,Kesesuaian,Eligible Chatbot,Weekend,Summary LLAMA3.1,Pertanyaan,Jawaban
0,28,113705,Whatsapp,2024-09-29 17:40,2024-10-01 06:22,2024-10-01 07:44,2024-10-01 13:38,2024-10-01 07:45,Angga Kusumawijaya,Grup 1,...,tidak jelas,Information,"Pelanggan, Bapak Adnan, menghubungi layanan pe...",Belum Terselesaikan,Sesuai,No,False,Pelanggan Bapak Adnan menghubungi layanan cust...,Setelah menganalisis percakapan antara custome...,** No data
1,34,113712,Whatsapp,2024-09-27 16:26,2024-09-27 19:23,2024-10-01 08:14,2024-10-03 08:56,2024-10-01 08:17,Adhardiansyah,Grup 1,...,lainnya,Incident,"Pelanggan, Bapak Afif, menghadapi masalah deng...",Belum Terselesaikan,Tidak Sesuai,No,False,Pelanggan Bapak Afif menghubungi layanan custo...,Setelah menganalisis percakapan antara custome...,** Kami akan segera melakukan pengecekan terka...
2,35,113713,Whatsapp,2024-08-13 10:25,2024-08-22 15:28,2024-10-01 08:16,2024-10-01 10:13,2024-10-01 08:18,Angga Kusumawijaya,Grup 1,...,reset password,Incident,"Pelanggan, Victor Gosal, menghadapi masalah ke...",Belum Terselesaikan,Sesuai,No,False,Berikut adalah ringkasan dalam satu paragraf:\...,Setelah menganalisis percakapan antara custome...,**\n1. Silahkan koordinasikan dengan atasan un...
3,55,113724,Email,-,~,2024-10-01 09:04,2024-10-01 09:49,2024-10-01 09:04,Adhardiansyah,Grup 1,...,lainnya,Request,"Pelanggan, Ibu Syifa Lutfiyyah, mengajukan per...",Terselesaikan,Sesuai,No,False,Pelanggan menghubungi layanan customer service...,Setelah menganalisis percakapan antara custome...,** No data
4,73,113743,Whatsapp,-,~,2024-10-01 09:47,2024-10-02 00:30,2024-10-01 10:05,Rangga Firmansyah,Grup 1,...,lainnya,Incident,"Pelanggan, Bapak Karsiyanto, mengalami masalah...",Belum Terselesaikan,Tidak Sesuai,No,False,Berikut adalah ringkasan dalam satu paragraf:\...,Setelah menganalisis percakapan antara custome...,**\nMohon mengisi data berikut... (ini tidak s...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4174,32541,114052,Whatsapp,-,~,2024-12-31 16:20,2024-12-31 19:16,2024-12-31 16:45,Roy Adam,Grup 1,...,lainnya,Incident,"Pelanggan, Ahmad Faishal, melaporkan masalah k...",Terselesaikan,Sesuai,No,False,"Pelanggan, Ahmad Faishal, menghubungi layanan ...",Setelah menganalisis percakapan antara custome...,Baik Bapak/Ibu mengenai situasi yang dihadapi/...
4175,32555,114068,Whatsapp,2024-11-21 09:01,2024-11-21 10:28,2024-12-31 17:08,2024-12-31 18:23,2024-12-31 17:55,Zilliah Mankusa,Grup 1,...,lainnya,Incident,"Pelanggan, Velayati Annisa Fitri, mengalami ma...",Belum Terselesaikan,Sesuai,No,False,"Pelanggan, Ibu Velayati Annisa Fitri, menghubu...",Setelah menganalisis percakapan antara custome...,**\nMohon isikan kolom A s/d E pada file berik...
4176,32557,114071,Whatsapp,2024-12-28 15:08,2024-12-30 14:10,2024-12-31 17:25,2024-12-31 17:37,2024-12-31 17:25,Angga Kusumawijaya,Grup 1,...,tidak jelas,Information,"Pelanggan, Ibu Salma, bertanya tentang cara me...",Belum Terselesaikan,Tidak Sesuai,No,False,"Pelanggan, Ibu Salma, menghubungi layanan cust...",Setelah menganalisis percakapan antara custome...,"Mohon maaf Ibu Salma Guga, dikarenakan tidak a..."
4177,32558,114072,Whatsapp,2024-06-28 14:36,2024-06-28 19:29,2024-12-31 17:35,2024-12-31 18:10,2024-12-31 18:00,Zilliah Mankusa,Grup 1,...,epic,Incident,"Pelanggan, Pak Darmanto, menghadapi masalah di...",Belum Terselesaikan,Sesuai,No,False,"Pelanggan, Darmanto, menghubungi layanan custo...",Setelah menganalisis percakapan antara custome...,**


In [18]:
def clean_data(df):
    # Fungsi untuk membersihkan pertanyaan
    def clean_question(text):
        if not isinstance(text, str):
            return text

        text = text.replace('*', '').strip()  # Menghilangkan karakter '*'
        paragraphs = [p.strip() for p in text.split('\n') if p.strip()]  # Mengambil paragraf yang tidak kosong
        if not paragraphs:
            return text

        # Jika ada pola seperti 'Pertanyaan 1:' atau 'Pertanyaan x:' ambil isi setelahnya
        match = re.search(r'Pertanyaan\s*\d+:\s*(.*)', text, re.DOTALL)
        if match:
            question_text = match.group(1).strip()
            # Jika pertanyaan terpotong atau berlanjut, hanya ambil kalimat pertama
            question_text = question_text.split('\n')[0].strip()
            return question_text

        # Jika tidak ada pola tersebut, ambil paragraf terakhir
        last_paragraph = paragraphs[-1]

        # Menghilangkan nomor di awal paragraf
        last_paragraph = re.sub(r'^\d+\.\s*', '', last_paragraph)
        return last_paragraph.strip()

    # Fungsi untuk membersihkan jawaban
    def clean_answer(text):
        if not isinstance(text, str):
            return text

        text = text.replace('*', '').strip()  # Menghilangkan karakter '*'
        paragraphs = [p.strip() for p in text.split('\n') if p.strip()]  # Mengambil paragraf yang tidak kosong
        if not paragraphs:
            return text

        # Jika jawaban berupa daftar prosedur bernomor, gabungkan menjadi satu paragraf
        if re.match(r'^\d+\.\s', paragraphs[0]):
            combined = ' '.join(paragraphs)
            return combined.strip()

        first_paragraph = paragraphs[0]  # Mengambil paragraf pertama jika bukan prosedur
        return first_paragraph.strip()

    # Menerapkan fungsi pembersihan ke kolom
    df['Pertanyaan'] = df['Pertanyaan'].apply(clean_question)
    df['Jawaban'] = df['Jawaban'].apply(clean_answer)

    return df

In [19]:
clean_data(df)
df

,First Index,Interaction ID,Channel,Last room created,Last room resolved,Current room created,Current room resolved,Last Inbound Date,Assignee Agent Name,Assignee Agent Department,...,Symptom,Type,Summary GPT4o,Status Penyelesaian,Kesesuaian,Eligible Chatbot,Weekend,Summary LLAMA3.1,Pertanyaan,Jawaban
0,28,113705,Whatsapp,2024-09-29 17:40,2024-10-01 06:22,2024-10-01 07:44,2024-10-01 13:38,2024-10-01 07:45,Angga Kusumawijaya,Grup 1,...,tidak jelas,Information,"Pelanggan, Bapak Adnan, menghubungi layanan pe...",Belum Terselesaikan,Sesuai,No,False,Pelanggan Bapak Adnan menghubungi layanan cust...,No data,No data
1,34,113712,Whatsapp,2024-09-27 16:26,2024-09-27 19:23,2024-10-01 08:14,2024-10-03 08:56,2024-10-01 08:17,Adhardiansyah,Grup 1,...,lainnya,Incident,"Pelanggan, Bapak Afif, menghadapi masalah deng...",Belum Terselesaikan,Tidak Sesuai,No,False,Pelanggan Bapak Afif menghubungi layanan custo...,Jika kita ingin menulis jawaban berdasarkan pe...,Kami akan segera melakukan pengecekan terkait ...
2,35,113713,Whatsapp,2024-08-13 10:25,2024-08-22 15:28,2024-10-01 08:16,2024-10-01 10:13,2024-10-01 08:18,Angga Kusumawijaya,Grup 1,...,reset password,Incident,"Pelanggan, Victor Gosal, menghadapi masalah ke...",Belum Terselesaikan,Sesuai,No,False,Berikut adalah ringkasan dalam satu paragraf:\...,Bagaimana cara melakukan aktivasi NIK kembali?,1. Silahkan koordinasikan dengan atasan untuk ...
3,55,113724,Email,-,~,2024-10-01 09:04,2024-10-01 09:49,2024-10-01 09:04,Adhardiansyah,Grup 1,...,lainnya,Request,"Pelanggan, Ibu Syifa Lutfiyyah, mengajukan per...",Terselesaikan,Sesuai,No,False,Pelanggan menghubungi layanan customer service...,No data,No data
4,73,113743,Whatsapp,-,~,2024-10-01 09:47,2024-10-02 00:30,2024-10-01 10:05,Rangga Firmansyah,Grup 1,...,lainnya,Incident,"Pelanggan, Bapak Karsiyanto, mengalami masalah...",Belum Terselesaikan,Tidak Sesuai,No,False,Berikut adalah ringkasan dalam satu paragraf:\...,Bagaimana cara melakukan proses submit hasil POI?,Mohon mengisi data berikut... (ini tidak sesua...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4174,32541,114052,Whatsapp,-,~,2024-12-31 16:20,2024-12-31 19:16,2024-12-31 16:45,Roy Adam,Grup 1,...,lainnya,Incident,"Pelanggan, Ahmad Faishal, melaporkan masalah k...",Terselesaikan,Sesuai,No,False,"Pelanggan, Ahmad Faishal, menghubungi layanan ...",Koneksi internet tidak berfungsi?,Baik Bapak/Ibu mengenai situasi yang dihadapi/...
4175,32555,114068,Whatsapp,2024-11-21 09:01,2024-11-21 10:28,2024-12-31 17:08,2024-12-31 18:23,2024-12-31 17:55,Zilliah Mankusa,Grup 1,...,lainnya,Incident,"Pelanggan, Velayati Annisa Fitri, mengalami ma...",Belum Terselesaikan,Sesuai,No,False,"Pelanggan, Ibu Velayati Annisa Fitri, menghubu...",Bagaimana cara melakukan bulk create PDF MAPS?,"Mohon isikan kolom A s/d E pada file berikut, ..."
4176,32557,114071,Whatsapp,2024-12-28 15:08,2024-12-30 14:10,2024-12-31 17:25,2024-12-31 17:37,2024-12-31 17:25,Angga Kusumawijaya,Grup 1,...,tidak jelas,Information,"Pelanggan, Ibu Salma, bertanya tentang cara me...",Belum Terselesaikan,Tidak Sesuai,No,False,"Pelanggan, Ibu Salma, menghubungi layanan cust...",Bagaimana saya bisa mendapatkan link download ...,"Mohon maaf Ibu Salma Guga, dikarenakan tidak a..."
4177,32558,114072,Whatsapp,2024-06-28 14:36,2024-06-28 19:29,2024-12-31 17:35,2024-12-31 18:10,2024-12-31 18:00,Zilliah Mankusa,Grup 1,...,epic,Incident,"Pelanggan, Pak Darmanto, menghadapi masalah di...",Belum Terselesaikan,Sesuai,No,False,"Pelanggan, Darmanto, menghubungi layanan custo...",Apa yang harus dilakukan jika tombol Upload Fi...,


In [ ]:
df.to_csv("test_mytens1_cleaned.csv", index=False)

In [15]:
# Menghapus baris yang berisi "No Data"
df = df[(df['Pertanyaan'] != 'No data') & (df['Jawaban'] != 'No data')]

# Reset indeks
df = df.reset_index(drop=True)

# Menampilkan dataframe yang telah dibersihkan
df

,First Index,Interaction ID,Channel,Last room created,Last room resolved,Current room created,Current room resolved,Assignee Agent Name,Assignee Agent Department,Resolved Agent Name,...,Conversation,Produk,Symptom,Type,Summary,Status Penyelesaian,Kesesuaian,Eligible Chatbot,Pertanyaan,Jawaban
0,47,113715,Whatsapp,-,~,2024-10-01 08:50,2024-10-02 09:46,Fidlin Aulia,Assurance Product Operation - OCA,Fidlin Aulia,...,"""customer : 3""\n\nagent : mohon tunggu, oca re...",oca,oca->pelanggan tidak bisa login,Incident,"Pelanggan, Arief dari Humas Universitas Tadula...",Terselesaikan,Sesuai,No,Bagaimana cara melakukan blast ke 220-240 nomo...,Anda dapat melakukan blast dengan jumlah yang ...
1,57,113726,Whatsapp,2024-09-30 21:56,2024-09-30 22:20,2024-10-01 09:04,2024-10-01 11:25,Rezi Eka Putri,Assurance Product Operation - OCA,Rezi Eka Putri,...,"""customer : 3""\n\nagent : mohon tunggu, oca re...",oca,oca->terdapat kegagalan variable sehingga meny...,Request,"Dalam percakapan ini, pelanggan, Bapak Ferry, ...",Sebagian Terselesaikan,Tidak Sesuai,No,Bisa bantu saya untuk pendaftaran akunnya?,"Baik sebelumnya mohon maaf, jika berkenan sila..."
2,181,113377,Whatsapp,-,~,2024-10-01 13:19,2024-10-01 13:45,Nimatul HA,Grup 2,Nimatul HA,...,"""customer : 9""\n\nagent : mohon menunggu sesaa...",oca,oca->user not response,Information,Pelanggan menghubungi agen untuk meminta dafta...,Belum Terselesaikan,Sesuai,No,Apakah ada yang bisa kami bantu?,"Selamat siang Bapak/Ibu, terima kasih telah me..."
3,204,113231,Email,2024-09-30 14:37,2024-09-30 14:41,2024-10-01 14:08,2024-10-01 14:09,Danang Aristya Wiebowo,Grup 1,Danang Aristya Wiebowo,...,"""customer : dear mba rindy,\n\n\n\n\nberikut y...",oca,oca->template/template code tidak sesuai,Request,"Pelanggan, Panji Kukuh Priambodho, menghadapi ...",Sebagian Terselesaikan,Sesuai,No,Bagaimana saya bisa mendaftar untuk layanan Anda?,"Untuk mendaftar, silakan kunjungi situs web ka..."
4,300,113252,Whatsapp,-,~,2024-10-01 19:17,2024-10-01 20:53,Nofvel Maulana,Grup 2,Nofvel Maulana,...,"""customer : cs""\n\nagent : mohon menunggu sesa...",oca,oca->info layanan oca,Information,"Pelanggan, Syadewo Putra dari Royal King, meng...",Sebagian Terselesaikan,Tidak Sesuai,No,Apakah syarat dan ketentuannya?,"Baik sebelumnya mohon maaf, jika berkenan sila..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477,30675,113937,Whatsapp,2024-12-24 15:30,2024-12-24 15:40,2024-12-26 06:42,2024-12-26 06:56,Mochammad Chana El Achyar,Grup 2,Mochammad Chana El Achyar,...,"""customer : pengajuan layanan oca""\n\nagent : ...",oca,t-money->user tidak merespon,Information,Pelanggan menghubungi layanan OCA dengan permi...,Belum Terselesaikan,Tidak Sesuai,No,Apakah ada?,"Baik Anda, kami siap membantu menyelesaikan pe..."
478,31159,113856,Whatsapp,-,~,2024-12-27 14:19,2024-12-27 14:46,Anggita Windiana Sari,Grup 2,Anggita Windiana Sari,...,"""customer : pengajuan layanan oca""\n\nagent : ...",oca,t-money->salah sambung,Incident,"Pelanggan, Sardani, melaporkan masalah terkait...",Belum Terselesaikan,Sesuai,No,Apakah informasi layanan OCA - C11DA20E - EMPTY?,Mohon maaf saat ini Anda terhubung dengan laya...
479,31553,113255,Whatsapp,-,~,2024-12-28 19:43,2024-12-28 19:59,Sesep Irawan,Grup 2,Sesep Irawan,...,"""customer : pengajuan layanan oca""\n\nagent : ...",oca,t-money->user tidak merespon,Request,Pelanggan menghubungi layanan OCA namun tidak ...,Belum Terselesaikan,Tidak Sesuai,No,"""pengajuan layanan oca""","""mohon tunggu, oca representative akan segera ..."
480,31678,113803,Whatsapp,-,~,2024-12-29 15:07,2024-12-29 15:13,Budi Yulianto,Grup 2,Budi Yulianto,...,"""customer : pengajuan layanan oca""\n\nagent : ...",oca,tidak jelas,Anomaly,"Dalam percakapan ini, pelanggan tidak mengungk...",tidak jelas,tidak jelas,No,Apakah ada yang bisa kami bantu?,"Baik bapak/ibu, apabila tidak ada yang dapat k..."


In [16]:
df.to_csv("OCA_llama_clean.csv", index=False)